## Q1

In [ ]:
class GoalBasedAgent:
    def __init__(self, goal):
        self.goal = goal

    def formulate_goal(self, percept):
        if percept == self.goal:
            return 'Goal Reached'
        return "Searching"
    
    def dfs(self, graph, start):
        visited = []
        stack = []

        visited.append(start)
        stack.append(start)

        while stack:
            node = stack.pop()
            print(node, end=" ")

            if node == self.goal:
                return f"Goal {self.goal} found"
            
            for neighbour, cost in reversed(graph[node].items()):
                if neighbour not in visited:
                    visited.append(neighbour)
                    stack.append(neighbour)
        return f"Goal {self.goal} not found!"
    
    def dls(self, start, graph, depth_limit):
        visited = []
        def dfs(node, depth):
            if depth > depth_limit:
                return None
            
            visited.append(node)
            if node == self.goal:
                return f"Goal {self.goal} not found! Path: {visited}"
            
            for neighbour, cost in graph[node].items():
                if neighbour not in visited:
                    path = dfs(neighbour, depth + 1)
                if path:
                    return path
            visited.pop()
            return None
        return dfs(start, 0)
    
    def act(self, percept, graph, type):
        goal_status = self.formulate_goal(percept)
        if goal_status == 'Goal Reached':
            return 'Goal Reached'
        elif type == 'dfs':
            return self.dfs(graph, percept)
        elif type == 'dls':
            return self.dls(percept, graph, 4)

class UtilityBasedAgent:
    def __init__(self, goal):
        self.goal = goal

    def formulate_goal(self, percept):
        if percept == self.goal:
            return 'Goal Reached'
        return "Searching"
    
    def ucs(self, start, graph):
        frontier = [(start, 0)]
        visited = set()
        came_from = {start: None}
        cost_so_far = {start: 0}

        while frontier:
            frontier.sort(key=lambda x: x[1])

            current_node, current_cost = frontier.pop(0)

            if current_node in visited:
                continue

            visited.add(current_node)

            if current_node == self.goal:
                path = []
                while current_node is not None:
                    path.append(current_node)
                    current_node = came_from[current_node]
                path.reverse()
                return f"Goal Found! Path: {path}, Total Cost: {current_cost}"
            
            for node, cost in graph[current_node].items():
                new_cost = current_cost + cost
                if node not in cost_so_far or new_cost < cost_so_far[node]:
                    cost_so_far[node] = new_cost
                    came_from[node] = current_node
                    frontier.append((node, new_cost))
        return f"Goal Not Found!"

    def act(self, percept, graph):
        goal_status = self.formulate_goal(percept)
        if goal_status == 'Goal Reached':
            return 'Goal Reached'
        return self.ucs(percept, graph)

class Environment:
    def __init__(self, graph):
        self.graph = graph
    
    def get_percept(self, node):
        return node
    
    def get_graph(self):
        return self.graph

def run_goal_agent(agent: GoalBasedAgent, environment: Environment, start_node, type):
    percept = environment.get_percept(start_node)
    action = agent.act(percept, environment.get_graph(), type)
    print(action)
    
def run_utility_agent(agent: UtilityBasedAgent, environment: Environment, start_node):
    percept = environment.get_percept(start_node)
    action = agent.act(percept, environment.get_graph())
    print(action)
    



# Q2

In [ ]:
def minimum_cost_tsp(graph):
    n = len(graph)
    
    def dfs(start, current, visited_count, current_cost, visited): 
        if visited_count == n:
            for neighbor, cost in graph[current]:
                if neighbor == start:
                    return current_cost + cost
            return float('inf')
        
        min_cost = float('inf')
        for neighbor, cost in graph[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                min_cost = min(
                    min_cost, 
                    dfs(start, neighbor, visited_count + 1, current_cost + cost, visited)
                )
                visited.remove(neighbor) 
        
        return min_cost
    
    min_tour_cost = float('inf')
    for city in graph:
        min_tour_cost = min(min_tour_cost, dfs(city, city, 1, 0, {city}))
    
    return min_tour_cost

graph = {
    1: [(2, 10), (4, 20), (3, 15)],
    2: [(1, 10), (4, 25), (3, 35)],
    3: [(2, 35), (4, 30), (1, 15)],
    4: [(1, 20), (2, 25), (3, 30)]
}

print(minimum_cost_tsp(graph))

80


# Q3

In [7]:
# Iterative Deepening

def dls(graph, node, goal, depth, path):
    if depth == 0:
        return False
    
    if node == goal:
        path.append(node)
        return True
    
    if node not in graph:
        return False
    
    for neighbour in graph[node]:
        if dls(graph, neighbour, goal, depth - 1, path):
            path.append(node)
            return True
    return False

def iterative_deepening(graph, start, goal, max_depth):
    for depth in range(max_depth + 1):
        print(f"Depth: {depth}")
        path = []
        if dls(graph, start, goal, depth, path):
            path.reverse()
            print(f"Path To Goal: {path}")
            return
    print("No Path Found")


graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': ['H'],
    'E': [],
    'F': ['I'],
    'G': [],
    'H': [],
    'I': []
}

iterative_deepening(graph, 'A', 'I', 5)

Depth: 0
Depth: 1
Depth: 2
Depth: 3
Depth: 4
Path To Goal: ['A', 'C', 'F', 'I']


In [10]:
# Bidirectional Search

def bfs(graph, queue, visited, other_visited):
    current = queue.pop(0)

    for neighbour in graph[current]:
        if neighbour not in visited:
            queue.append(neighbour)
            visited[neighbour] = current
            if neighbour in other_visited:
                return True
    return False

def construct_path(forward_visited, backward_visited):
    intersection = set(forward_visited) & set(backward_visited)
    if not intersection:
        return None
    
    meet_point = intersection.pop()

    path = []
    node = meet_point
    while node is not None:
        path.append(node)
        node = forward_visited[node]
    path.reverse()

    node = backward_visited[meet_point]
    while node is not None:
        path.append(node)
        node = backward_visited[node]
    
    return path

def bidirectional_search(graph, start, goal):
    if start == goal:
        return [start]
    
    forward_queue = [start]
    backward_queue = [goal]

    forward_visited = {start: None}
    backward_visited = {goal: None}

    while forward_queue and backward_queue:
        if bfs(graph, forward_queue, forward_visited, backward_visited):
            return construct_path(forward_visited, backward_visited)
        
        if bfs(graph, backward_queue, backward_visited, forward_visited):
            return construct_path(forward_visited, backward_visited)
        
    return None

graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': ['H'],
    'E': [],
    'F': ['I'],
    'G': [],
    'H': [],
    'I': []
}

start = 'A'
goal = 'I'
path = bidirectional_search(graph, start, goal)
print("Path from", start, "to", goal, ":", path)


Path from A to I : None
